#Topic_modeling(LDA) 직접 구현

###문서 정의(문장 구분 포함)


In [1]:
docs = ['cute kitty',
          'eat rice cake',
          'kitty hamster',
          'eat bread',
          'rice bread cake',
          'cute hamster eat bread cake']
        

###토큰화

In [2]:
doc_ls = [doc.split() for doc in docs]
doc_ls

[['cute', 'kitty'],
 ['eat', 'rice', 'cake'],
 ['kitty', 'hamster'],
 ['eat', 'bread'],
 ['rice', 'bread', 'cake'],
 ['cute', 'hamster', 'eat', 'bread', 'cake']]

###토큰 인덱싱 딕셔너리 생성


In [3]:
# 토큰의 인덱스 생성
def word2id(doc_ls) :
    from collections import defaultdict

    word2id = defaultdict(lambda : len(word2id))
    [word2id[token] for doc in doc_ls for token in doc ]     

    return word2id

# 토큰의 인덱스를 키로 갖는 사전 생성
def id2word(word2id) :

    id2word ={}
    for k,v in word2id.items():
        id2word[v] = k
    return id2word

In [4]:
word2idx = word2id(doc_ls)

In [5]:
idex2word = id2word(word2idx)

In [6]:
word2idx

defaultdict(<function __main__.word2id.<locals>.<lambda>>,
            {'bread': 6,
             'cake': 4,
             'cute': 0,
             'eat': 2,
             'hamster': 5,
             'kitty': 1,
             'rice': 3})

In [7]:
idex2word

{0: 'cute',
 1: 'kitty',
 2: 'eat',
 3: 'rice',
 4: 'cake',
 5: 'hamster',
 6: 'bread'}

###각 단어에 랜덤하게 토픽 할당(토픽 개수 임의 지정)

In [8]:
import random

# 토픽의 개수를 정한뒤, 각 단어에 랜덤하게 토픽 할당
topic_num = 2

# 2차원 리스트 그대로 쓰자
# doc_ls와 같은 모양의 리스트 생성

topic_ls = []

for doc_idx in range(len(doc_ls)) :
    temp_ls = []
    for i in range(len(doc_ls[doc_idx])):
        temp_ls.append(random.randint(1,topic_num))
    topic_ls.append(temp_ls)

topic_ls



[[1, 2], [1, 2, 2], [1, 2], [1, 1], [2, 1, 1], [2, 2, 2, 2, 2]]

###문서별 토픽 분포 표 작성

In [9]:
import numpy as np
topic_dist_matrix_doc = np.ones((topic_num, len(doc_ls))) *0.1 #행: 토픽 수, 열 : 문서 수 //0.1은 a

#문서별 토픽 분포 확인하고 topic_dist_matrix에 반영해주기
for idx, doc in enumerate(topic_ls):
    for word_topic in doc :
        topic_dist_matrix_doc[word_topic-1][idx] += 1
topic_dist_matrix_doc


array([[1.1, 1.1, 1.1, 2.1, 2.1, 0.1],
       [1.1, 2.1, 1.1, 0.1, 1.1, 5.1]])

In [10]:
import pandas as pd

matrix_doc = pd.DataFrame(topic_dist_matrix_doc,columns=['문서 {}'.format(i) for i in range(len(doc_ls))], index= ['topic {}'.format(j+1) for j in range(topic_num)])
matrix_doc

,문서 0,문서 1,문서 2,문서 3,문서 4,문서 5
topic 1,1.1,1.1,1.1,2.1,2.1,0.1
topic 2,1.1,2.1,1.1,0.1,1.1,5.1


###토픽 내 단어 분포 매트릭스

In [11]:
#토픽 내 단어 분포 표 초기화
topic_dist_matrix_word = np.ones((topic_num, len(word2idx))) *0.001 #행: 토픽 수, 열 : 유니크 토큰 수    0.01은 b

#토픽 내 단어 분포 표 # 각 위치에 해당하는 토픽을 받고 doc_ls
for idx, doc in enumerate(topic_ls):
    for idx2, word_topic in enumerate(doc) :
        topic_dist_matrix_word[word_topic-1][word2idx[doc_ls[idx][idx2]]] += 1 # doc_ls[idx][idx2]는 해당 토픽이 배정된 단어
print(word2idx)
print(topic_dist_matrix_word)

defaultdict(<function word2id.<locals>.<lambda> at 0x7f29afd566a8>, {'cute': 0, 'kitty': 1, 'eat': 2, 'rice': 3, 'cake': 4, 'hamster': 5, 'bread': 6})
[[1.001e+00 1.001e+00 2.001e+00 1.000e-03 1.001e+00 1.000e-03 2.001e+00]
 [1.001e+00 1.001e+00 1.001e+00 2.001e+00 2.001e+00 2.001e+00 1.001e+00]]


In [12]:
import pandas as pd

matrix_word = pd.DataFrame(topic_dist_matrix_word,columns=[i for i in word2idx.keys()], index= ['topic {}'.format(j+1) for j in range(topic_num)])
matrix_word

,cute,kitty,eat,rice,cake,hamster,bread
topic 1,1.001,1.001,2.001,0.001,1.001,0.001,2.001
topic 2,1.001,1.001,1.001,2.001,2.001,2.001,1.001


In [13]:
# 문서A 내에 topic 1이 있을 확률

#문서 내 토픽 합계 리스트
topic_sum_ls = []
for j in range(len(doc_ls)) :
    temp_sum = 0
    for i in range(topic_num) :
        temp_sum += topic_dist_matrix_doc[i][j]
    topic_sum_ls.append(temp_sum)
topic_sum_ls

[2.2, 3.2, 2.2, 2.2, 3.2, 5.199999999999999]

In [14]:
matrix_doc.loc['합계'] = topic_sum_ls
matrix_doc

,문서 0,문서 1,문서 2,문서 3,문서 4,문서 5
topic 1,1.1,1.1,1.1,2.1,2.1,0.1
topic 2,1.1,2.1,1.1,0.1,1.1,5.1
합계,2.2,3.2,2.2,2.2,3.2,5.2


###문서 내 토픽이 있을 확률

In [15]:
# 문서 내에 토픽이 있을 확률을 나타내는 새로운 매트릭스 생성
topic_in_doc_matrix = np.zeros((topic_num, len(doc_ls)))

for i in range(topic_num) :
    for j in range(len(doc_ls)) :
        topic_in_doc_matrix[i][j] = topic_dist_matrix_doc[i][j]/topic_sum_ls[j]
        # topic_in_doc_matrix[i][j] = 0.001/topic_sum_ls[j]
topic_in_doc_matrix
pd.DataFrame(topic_in_doc_matrix,columns=['문서 {}'.format(i) for i in range(len(doc_ls))], index= ['topic {}'.format(j+1) for j in range(topic_num)])

,문서 0,문서 1,문서 2,문서 3,문서 4,문서 5
topic 1,0.5,0.34375,0.5,0.954545,0.65625,0.019231
topic 2,0.5,0.65625,0.5,0.045455,0.34375,0.980769


###토픽 내의 단어가 특정 단어일 확률

In [16]:
word_in_topic_matrix = np.zeros((topic_num, len(word2idx)))

for idx, topic in enumerate(topic_dist_matrix_word) :
    for idx2, topic_word in enumerate(topic) :
        word_in_topic_matrix[idx][idx2] = topic_word/sum(topic)
word_in_topic_matrix
pd.DataFrame(word_in_topic_matrix, columns= word2idx.keys(), index= ['topic {}'.format(j+1) for j in range(topic_num)])

,cute,kitty,eat,rice,cake,hamster,bread
topic 1,0.142857,0.142857,0.285572,0.000143,0.142857,0.000143,0.285572
topic 2,0.100030,0.100030,0.100030,0.199960,0.199960,0.199960,0.100030


###토픽 할당

In [18]:
#단어마다 토픽을 할당해주기 위해 doc_ls와 같은 형식의 topic_distribution_ls 초기화
topic_distribution_ls = [[0 for x in range(len(topic_ls[i]))] for i in range(len(topic_ls))]

#문서 안의 특정 단어가 토픽 1일 확률
#doc_ls a문서의 첫번째 단어가 토픽 1일 확률, 토픽 2일 확률을 비교해 큰값 doc_ls 형태에 반영
for idx, doc in enumerate(topic_ls):
    for idx2, word_topic in enumerate(doc) :
        topic_dist_matrix_word[word_topic-1][word2idx[doc_ls[idx][idx2]]] += 1

compare_list = [] 
for idx, doc in enumerate(doc_ls) : #문서 인덱스 가져오기
    for idx2, word_topic in enumerate(doc) :
        temp_list = []
        for i in range(topic_num) :
            temp_list.append(topic_in_doc_matrix[i][idx] * word_in_topic_matrix[i][word2idx[doc_ls[idx][idx2]]])   #토픽 1일 확률과 토픽 2일 확률 비교
        compare_list.append(temp_list)

        # compare_list[idx].argsort()
        topic_distribution_ls[idx][idx2] = np.array(compare_list[idx]).argsort()[0] + 1 # topic 분류를 위해 1 더함

# print(compare_list)
# print()
# print(topic_distribution_ls)
topic_distribution_ls


[[2, 2], [2, 2, 2], [2, 2], [1, 1], [1, 1, 1], [2, 2, 2, 2, 2]]

In [19]:
#topic_distribution_ls 을 topic_ls에 넣고 다시 전체 반복 

###함수화

In [20]:
class LDA :
    def __init__(self, topic_num, doc_ls) :
        self.doc_ls = doc_ls
        self.topic_num = topic_num

    def word2id(self) :
        from collections import defaultdict

        word2id = defaultdict(lambda : len(word2id))
        [word2id[token] for doc in self.doc_ls for token in doc ]     

        return word2id

    # 토큰의 인덱스를 키로 갖는 사전 생성
    def id2word(word2id) :

        id2word ={}
        for k,v in word2id.items():
            id2word[v] = k
        return id2word

    def mk_topic_dist_matrix(self, topic_ls) :
        import numpy as np
        topic_dist_matrix_doc = np.ones((self.topic_num, len(self.doc_ls))) *0.1 #행: 토픽 수, 열 : 문서 수 //0.1은 a

        #문서별 토픽 분포 확인하고 topic_dist_matrix에 반영해주기
        for idx, doc in enumerate(topic_ls):
            for word_topic in doc :
                topic_dist_matrix_doc[word_topic-1][idx] += 1
        return topic_dist_matrix_doc

    def mk_topic_dist_matrix_word(self, topic_ls, word2idx) :
        #토픽 내 단어 분포 표 초기화
        topic_dist_matrix_word = np.ones((self.topic_num, len(word2idx))) *0.001 #행: 토픽 수, 열 : 유니크 토큰 수    0.01은 b

        #토픽 내 단어 분포 표 # 각 위치에 해당하는 토픽을 받고 doc_ls
        for idx, doc in enumerate(topic_ls):
            for idx2, word_topic in enumerate(doc) :
                topic_dist_matrix_word[word_topic-1][word2idx[self.doc_ls[idx][idx2]]] += 1 # doc_ls[idx][idx2]는 해당 토픽이 배정된 단어

        return topic_dist_matrix_word

    def chance_topic_in_doc(self, topic_dist_matrix_doc) :
        # 문서A 내에 topic 1이 있을 확률

        #문서 내 토픽 합계 리스트
        topic_sum_ls = []
        for j in range(len(self.doc_ls)) :
            temp_sum = 0
            for i in range(self.topic_num) :
                temp_sum += topic_dist_matrix_doc[i][j]
            topic_sum_ls.append(temp_sum)

        # 문서 내에 토픽이 있을 확률을 나타내는 새로운 매트릭스 생성
        topic_in_doc_matrix = np.zeros((self.topic_num, len(self.doc_ls)))

        for i in range(self.topic_num) :
            for j in range(len(self.doc_ls)) :
                topic_in_doc_matrix[i][j] = topic_dist_matrix_doc[i][j]/topic_sum_ls[j]
        
        return topic_in_doc_matrix
    
    def mk_word_in_topic_matrix(self, word2idx, topic_dist_matrix_word) :
        word_in_topic_matrix = np.zeros((self.topic_num, len(word2idx)))

        # topic 내의 단어가 cute일 확률
        for idx, topic in enumerate(topic_dist_matrix_word) :
            for idx2, topic_word in enumerate(topic) :
                word_in_topic_matrix[idx][idx2] = topic_word/sum(topic)
        return word_in_topic_matrix

    def mk_topic_distribution_ls(self, topic_ls, topic_in_doc_matrix, word_in_topic_matrix, topic_dist_matrix_word) :

        #문서 내 토픽 분포 리스트 초기화
        topic_distribution_ls = [[0 for x in range(len(topic_ls[i]))] for i in range(len(topic_ls))]

        #문서 안의 특정 단어가 토픽 1일 확률
        #doc_ls a문서의 첫번째 단어가 토픽 1일 확률, 토픽 2일 확률을 비교해 큰값 doc_ls 형태에 반영

        compare_list = [] 
        for idx, doc in enumerate(self.doc_ls) : #문서 인덱스 가져오기
            for idx2, word_topic in enumerate(doc) :
                temp_list = []
                for i in range(self.topic_num) :
                    temp_list.append(topic_in_doc_matrix[i][idx] * word_in_topic_matrix[i][word2idx[doc_ls[idx][idx2]]])   #토픽 1일 확률과 토픽 2일 확률 비교
                compare_list.append(temp_list)

                # compare_list[idx].argsort()
                topic_distribution_ls[idx][idx2] = np.array(compare_list[idx]).argsort()[0] + 1 # topic 분류를 위해 1 더함

        # print(compare_list)
        # print()
        # print(topic_distribution_ls)
        return topic_distribution_ls

    def mk_random_topic_ls(self):
        topic_ls = []

        for doc_idx in range(len(self.doc_ls)) :
            temp_ls = []
            for i in range(len(self.doc_ls[doc_idx])):
                temp_ls.append(random.randint(1, self.topic_num))
            topic_ls.append(temp_ls)

        return topic_ls

    def mk_final_topic_ls(self, topic_ls, word2idx) :
        for x in range(100) :
            temp_topic_ls = topic_ls

            topic_dist_matrix_doc = self.mk_topic_dist_matrix(topic_ls)
            topic_dist_matrix_word = self.mk_topic_dist_matrix_word(topic_ls, word2idx)
            topic_in_doc_matrix = self.chance_topic_in_doc(topic_dist_matrix_doc)
            word_in_topic_matrix = self.mk_word_in_topic_matrix(word2idx, topic_dist_matrix_word)
            topic_ls = self.mk_topic_distribution_ls(topic_ls,topic_in_doc_matrix, word_in_topic_matrix,topic_dist_matrix_word)

            # print(topic_ls)
            if temp_topic_ls == topic_ls :
                break
        print('최종 분류 토픽 : {}'.format(topic_ls))
        return topic_ls
    
    def topic_modeling(self, topic_ls, word2idx, idex2word) :
        for idx, topic in enumerate(word_in_topic_matrix) :
            print("Topic {}".format(idx+1), end = ' : ')
            for i in range(len(word2idx)) :
                temp_idx = topic.argsort()[::-1][i]
                print(idex2word[temp_idx], topic[temp_idx].round(4),end = ', ')
            print()
        


###LDA클래스 실행

In [21]:
test = LDA(3,doc_ls)
topic_ls = test.mk_random_topic_ls()
print("랜덤 토픽 할당 : {}".format(topic_ls))
topic_ls = test.mk_final_topic_ls(topic_ls,word2idx)


test.topic_modeling(topic_ls,word2idx,idex2word)


랜덤 토픽 할당 : [[1, 3], [2, 2, 2], [2, 3], [1, 3], [1, 3, 1], [3, 2, 3, 2, 3]]
최종 분류 토픽 : [[3, 3], [1, 1, 1], [2, 2], [2, 2], [2, 2, 2], [1, 1, 1, 1, 1]]
Topic 1 : bread 0.2856, eat 0.2856, cake 0.1429, kitty 0.1429, cute 0.1429, hamster 0.0001, rice 0.0001, 
Topic 2 : hamster 0.2, cake 0.2, rice 0.2, bread 0.1, eat 0.1, kitty 0.1, cute 0.1, 


###To_Do_fix :
1. 단어마다 토픽 재 할당 해줄때, 이미 해당 단어의 토픽 제외
2. 토픽 할당 시 결과값 진동 